In [1]:
import requests
import re
import shutil
import os

from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

from datetime import datetime
from time import sleep

import urllib.request

In [2]:
with open('batches/batch.txt','r') as f:
    record = eval(f.read())

In [3]:
ds_dir = 'training_data'
if not os.path.exists(ds_dir):
    os.mkdir(ds_dir)

In [4]:
def download_image(img_url, path):
    r = requests.get(img_url, stream=True)
    if r.status_code == 200:
        with open(path, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 
    else:
        print("Image not there")
        cd = input()
        if cd == "1":
            return
        raise Exception(f"Image {img_url} not there")
            
def click(driver,elem):
    action = ActionChains(driver)
    action.click(on_element=elem)
    action.perform()

def scroll(driver):
    for i in range(33):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(1)
        c = 0
        while True:
            try:
                elem = driver.find_element(By.XPATH, '//*[@id="content"]/div/div/form/div[3]/button')
                click(driver,elem)
                sleep(1)
                break
            except:
                if c <= 15:
                    c+=1
                    sleep(0.5)
                    continue
                else:
                    log("Couldn't find more button")
                    return  

def gettime():
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                
def log(message):
    with open('log.txt','a') as f:
        f.write(f'{gettime()} - {message}\n')

In [5]:
driver = webdriver.Chrome()

with open('log.txt','w') as f:
    f.write("")

rec = record[:]
pbar = tqdm(record)
for sp, code in pbar:
    pbar.set_description(f'Processing {sp}')
    q=0
    while q==0: 
        fp = os.path.join(ds_dir,sp)
        link = f'https://media.ebird.org/catalog?taxonCode={code}&mediaType=photo&view=list&sort=rating_rank_desc'
        
        while True:
            try:
                driver.get(link)
                break
            except:
                log("Couldn't load link")
                sleep(.5)
                continue
        
        while True:
            try:
                num = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[1]/main/div/div/form/div[1]/div[2]/div[1]/a[1]/span/span[2]")
                num = int(num.text.replace(",",""))

                num = num if num < 1000 else 1000
                break
            except Exception as e:
                log(f"Couldn't get count : {e}")
                sleep(.5)
                continue
                  
        scroll(driver)

        photos = driver.find_elements(By.XPATH, '//div[@class="ResultsList-media-image"]/img')
#         ml = driver.find_elements(By.XPATH, '//div[@class="ResultsList-header"]/a[1]') 
        
        if len(photos) > 1000:
            photos = photos[:1000]
#             ml = ml[:1000]

        n=1        
        if len(photos) >= num:
            q=1
        else:
            continue
            
        imgs=list()
        for i in photos[:num]:
            c=0
            try:
                img = i.get_attribute('src')
            except:
                q=0
                log("Couldn't get image link")
                sleep(.5)
                continue
#             j = photos.index(i)
#             path = os.path.join(fp, ml[j].text.strip()+'.jpg')
            imgs.append(img)

    with open(f'image_links/{sp}.txt','w') as f:
        f.write(str(imgs))
        
    rec.remove((sp,code))
    with open('batches/batch.txt','w') as f:
        f.write(str(rec))

Processing Green Imperial-Pigeon: 100%|█████████████████████████████████████████████████| 1/1 [01:42<00:00, 102.47s/it]


In [6]:
pbar = tqdm(os.listdir('image_links'))
for sp in pbar:
    sp = sp[:-4]
    c=1

    path = os.path.join('image_links',sp) +'.txt'
    dir = os.path.join(ds_dir,sp)

    if not os.path.exists(dir):
        os.mkdir(dir)

    with open(path,'r') as f:
        r = eval(f.read())
        for image in r:
            pbar.set_description(f'Processing {sp}:{c}')
            while True:
                try:
                    download_image(image,os.path.join(dir,f'{c}.jpg'))
                    c+=1
                    break
                except:
                    log(f"Couldn't download {image} - {sp}")
                    continue
    os.remove(path)

Processing Green Imperial-Pigeon:1000: 100%|████████████████████████████████████████| 17/17 [2:34:17<00:00, 544.54s/it]
